<a href="https://colab.research.google.com/github/werent4/ML_notebooks/blob/main/heart_disease_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip install lazypredict

In [81]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.decomposition import PCA

In [82]:
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.00,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.00,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.00,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.50,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.00,Up,0


In [83]:
df.shape

(918, 12)

In [84]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,918.00,53.51,9.43,28.00,47.00,54.00,60.00,77.00
RestingBP,918.00,132.40,18.51,0.00,120.00,130.00,140.00,200.00
Cholesterol,918.00,198.80,109.38,0.00,173.25,223.00,267.00,603.00
FastingBS,918.00,0.23,0.42,0.00,0.00,0.00,0.00,1.00
MaxHR,918.00,136.81,25.46,60.00,120.00,138.00,156.00,202.00
Oldpeak,918.00,0.89,1.07,-2.60,0.00,0.60,1.50,6.20
HeartDisease,918.00,0.55,0.50,0.00,0.00,1.00,1.00,1.00


In [85]:
df[df.Cholesterol>(df.Cholesterol.mean()+3*df.Cholesterol.std())]


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.00,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.00,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.60,Flat,0


In [86]:
df1 = df[df.Cholesterol<=(df.Cholesterol.mean()+3*df.Cholesterol.std())]
df1.shape

(915, 12)

In [87]:
df2 = df1[df1.Oldpeak<=(df1.Oldpeak.mean()+3*df1.Oldpeak.std())]
df2.shape

(909, 12)

In [88]:
df3 = df2[df2.RestingBP<=(df2.RestingBP.mean()+3*df2.RestingBP.std())]
df3.shape

(902, 12)

In [89]:
df_no_out = df3.copy()
df_no_out.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.00,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.00,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.00,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.50,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.00,Up,0


In [90]:
df_no_out.Sex.unique()

array(['M', 'F'], dtype=object)

In [91]:
df_no_out.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [92]:
df_no_out.ChestPainType.unique()# dummies

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [93]:
df_no_out.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [94]:
df_no_out.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True)

df_no_out.Sex.replace(
    {
        'M': 0,
        'F': 1
    },
    inplace=True)

df_no_out.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True)
df_no_out.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,ATA,140,289,0,1,172,0,0.00,Up,0
1,49,1,NAP,160,180,0,1,156,0,1.00,Flat,1
2,37,0,ATA,130,283,0,2,98,0,0.00,Up,0
3,48,1,ASY,138,214,0,1,108,1,1.50,Flat,1
4,54,0,NAP,150,195,0,1,122,0,0.00,Up,0


In [95]:
df_no_out.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)
df_no_out.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,ATA,140,289,0,1,172,0,0.00,3,0
1,49,1,NAP,160,180,0,1,156,0,1.00,2,1
2,37,0,ATA,130,283,0,2,98,0,0.00,3,0
3,48,1,ASY,138,214,0,1,108,1,1.50,2,1
4,54,0,NAP,150,195,0,1,122,0,0.00,3,0


In [96]:
df_dumm = pd.get_dummies(df_no_out, drop_first= True)
df_dumm.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,0,140,289,0,1,172,0,0.00,3,0,1,0,0
1,49,1,160,180,0,1,156,0,1.00,2,1,0,1,0
2,37,0,130,283,0,2,98,0,0.00,3,0,1,0,0
3,48,1,138,214,0,1,108,1,1.50,2,1,0,0,0
4,54,0,150,195,0,1,122,0,0.00,3,0,0,1,0


In [97]:
X = df_dumm.drop('HeartDisease', axis = 1)
y = df_dumm.HeartDisease

In [98]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2 , random_state = 30)

In [100]:
lazy=LazyClassifier(verbose=1)
models,predict=lazy.fit(X_train, X_test, y_train,y_test)


  3%|▎         | 1/29 [00:00<00:03,  7.62it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.861878453038674, 'Balanced Accuracy': 0.8621489621489622, 'ROC AUC': 0.8621489621489621, 'F1 Score': 0.8615743042977738, 'Time taken': 0.13057732582092285}
{'Model': 'BaggingClassifier', 'Accuracy': 0.8397790055248618, 'Balanced Accuracy': 0.8397435897435898, 'ROC AUC': 0.8397435897435898, 'F1 Score': 0.8397692231275438, 'Time taken': 0.049736976623535156}
{'Model': 'BernoulliNB', 'Accuracy': 0.8342541436464088, 'Balanced Accuracy': 0.8344322344322345, 'ROC AUC': 0.8344322344322345, 'F1 Score': 0.834102250449604, 'Time taken': 0.014219522476196289}


 14%|█▍        | 4/29 [00:00<00:02, 10.97it/s]

{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.8342541436464088, 'Balanced Accuracy': 0.8347374847374848, 'ROC AUC': 0.8347374847374848, 'F1 Score': 0.8330315285270069, 'Time taken': 0.1798255443572998}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.7955801104972375, 'Balanced Accuracy': 0.7957875457875458, 'ROC AUC': 0.7957875457875458, 'F1 Score': 0.7953177534198008, 'Time taken': 0.016429662704467773}
{'Model': 'DummyClassifier', 'Accuracy': 0.4972375690607735, 'Balanced Accuracy': 0.5, 'ROC AUC': 0.5, 'F1 Score': 0.3302684960551263, 'Time taken': 0.013239145278930664}
{'Model': 'ExtraTreeClassifier', 'Accuracy': 0.8176795580110497, 'Balanced Accuracy': 0.8178266178266178, 'ROC AUC': 0.8178266178266178, 'F1 Score': 0.8175682002819376, 'Time taken': 0.013272762298583984}


 45%|████▍     | 13/29 [00:00<00:00, 18.92it/s]

...

{'Model': 'LinearSVC', 'Accuracy': 0.850828729281768, 'Balanced Accuracy': 0.8512210012210013, 'ROC AUC': 0.8512210012210012, 'F1 Score': 0.8501152756155189, 'Time taken': 0.07735300064086914}
{'Model': 'LogisticRegression', 'Accuracy': 0.8453038674033149, 'Balanced Accuracy': 0.8457264957264958, 'ROC AUC': 0.8457264957264956, 'F1 Score': 0.8444400144291925, 'Time taken': 0.02201390266418457}
{'Model': 'NearestCentroid', 'Accuracy': 0.8397790055248618, 'Balanced Accuracy': 0.8399267399267399, 'ROC AUC': 0.8399267399267399, 'F1 Score': 0.8396811457023088, 'Time taken': 0.04133248329162598}
{'Model': 'NuSVC', 'Accuracy': 0.856353591160221, 'Balanced Accuracy': 0.8567765567765568, 'ROC AUC': 0.8567765567765567, 'F1 Score': 0.8555514419699645, 'Time taken': 0.0659024715423584}
{'Model': 'PassiveAggressiveClassifier', 'Accuracy': 0.8232044198895028, 'Balanced Accuracy': 0.8235042735042735, 'ROC AUC': 0.8235042735042734, 'F1 Score': 0.8227175287570075, 'Time taken': 0.017822742462158203}
{'M

 90%|████████▉ | 26/29 [00:01<00:00, 20.53it/s]

{'Model': 'RandomForestClassifier', 'Accuracy': 0.850828729281768, 'Balanced Accuracy': 0.8511599511599511, 'ROC AUC': 0.851159951159951, 'F1 Score': 0.850326330709189, 'Time taken': 0.22245478630065918}
{'Model': 'RidgeClassifier', 'Accuracy': 0.8397790055248618, 'Balanced Accuracy': 0.8401709401709402, 'ROC AUC': 0.84017094017094, 'F1 Score': 0.8390127034388907, 'Time taken': 0.021641969680786133}
{'Model': 'RidgeClassifierCV', 'Accuracy': 0.8397790055248618, 'Balanced Accuracy': 0.8401709401709402, 'ROC AUC': 0.84017094017094, 'F1 Score': 0.8390127034388907, 'Time taken': 0.024266719818115234}
{'Model': 'SGDClassifier', 'Accuracy': 0.7624309392265194, 'Balanced Accuracy': 0.7624542124542124, 'ROC AUC': 0.7624542124542124, 'F1 Score': 0.7624309392265194, 'Time taken': 0.02257251739501953}
{'Model': 'SVC', 'Accuracy': 0.8674033149171271, 'Balanced Accuracy': 0.8677045177045177, 'ROC AUC': 0.8677045177045177, 'F1 Score': 0.8670381465677557, 'Time taken': 0.05522727966308594}


100%|██████████| 29/29 [00:01<00:00, 17.61it/s]

{'Model': 'XGBClassifier', 'Accuracy': 0.856353591160221, 'Balanced Accuracy': 0.8567155067155068, 'ROC AUC': 0.8567155067155067, 'F1 Score': 0.8557726681832953, 'Time taken': 0.1178596019744873}
[LightGBM] [Info] Number of positive: 405, number of negative: 316
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 375
[LightGBM] [Info] Number of data points in the train set: 721, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.561720 -> initscore=0.248145
[LightGBM] [Info] Start training from score 0.248145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

In [101]:
predict

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.87,0.87,0.87,0.87,0.06
AdaBoostClassifier,0.86,0.86,0.86,0.86,0.13
NuSVC,0.86,0.86,0.86,0.86,0.07
XGBClassifier,0.86,0.86,0.86,0.86,0.12
KNeighborsClassifier,0.86,0.86,0.86,0.86,0.03
LinearSVC,0.85,0.85,0.85,0.85,0.08
RandomForestClassifier,0.85,0.85,0.85,0.85,0.22
LGBMClassifier,0.85,0.85,0.85,0.84,0.09
LogisticRegression,0.85,0.85,0.85,0.84,0.02


In [107]:
pca = PCA(0.95)
X_pca = pca.fit_transform(X)
X_pca.shape

(902, 2)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size = 0.2 , random_state = 30)

In [109]:
lazy=LazyClassifier(verbose=1)
models,predict=lazy.fit(X_train, X_test, y_train,y_test)


 14%|█▍        | 4/29 [00:00<00:01, 17.54it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7237569060773481, 'Balanced Accuracy': 0.724053724053724, 'ROC AUC': 0.724053724053724, 'F1 Score': 0.7229961386828241, 'Time taken': 0.1183168888092041}
{'Model': 'BaggingClassifier', 'Accuracy': 0.6906077348066298, 'Balanced Accuracy': 0.6906593406593406, 'ROC AUC': 0.6906593406593406, 'F1 Score': 0.6905888463899514, 'Time taken': 0.0457456111907959}
{'Model': 'BernoulliNB', 'Accuracy': 0.6629834254143646, 'Balanced Accuracy': 0.6636141636141637, 'ROC AUC': 0.6636141636141637, 'F1 Score': 0.6586094270420657, 'Time taken': 0.012067794799804688}
{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.6961325966850829, 'Balanced Accuracy': 0.6963369963369963, 'ROC AUC': 0.6963369963369963, 'F1 Score': 0.6957426064348392, 'Time taken': 0.06760263442993164}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.6850828729281768, 'Balanced Accuracy': 0.685042735042735, 'ROC AUC': 0.685042735042735, 'F1 Score': 0.685063645457586, 'Time taken': 0.0128552913

 45%|████▍     | 13/29 [00:00<00:00, 23.72it/s]

{'Model': 'ExtraTreesClassifier', 'Accuracy': 0.7237569060773481, 'Balanced Accuracy': 0.723870573870574, 'ROC AUC': 0.7238705738705737, 'F1 Score': 0.7236556934146994, 'Time taken': 0.18362736701965332}
{'Model': 'GaussianNB', 'Accuracy': 0.7348066298342542, 'Balanced Accuracy': 0.7346153846153847, 'ROC AUC': 0.7346153846153846, 'F1 Score': 0.7344661394470274, 'Time taken': 0.013346433639526367}
{'Model': 'KNeighborsClassifier', 'Accuracy': 0.712707182320442, 'Balanced Accuracy': 0.713003663003663, 'ROC AUC': 0.7130036630036629, 'F1 Score': 0.7119159842301371, 'Time taken': 0.020663022994995117}
{'Model': 'LabelPropagation', 'Accuracy': 0.7071823204419889, 'Balanced Accuracy': 0.7074481074481074, 'ROC AUC': 0.7074481074481074, 'F1 Score': 0.7065375251112803, 'Time taken': 0.03827190399169922}
{'Model': 'LabelSpreading', 'Accuracy': 0.712707182320442, 'Balanced Accuracy': 0.713003663003663, 'ROC AUC': 0.7130036630036629, 'F1 Score': 0.7119159842301371, 'Time taken': 0.05288553237915039

 59%|█████▊    | 17/29 [00:00<00:00, 27.13it/s]

{'Model': 'LogisticRegression', 'Accuracy': 0.6961325966850829, 'Balanced Accuracy': 0.6963369963369963, 'ROC AUC': 0.6963369963369963, 'F1 Score': 0.6957426064348392, 'Time taken': 0.019693374633789062}
{'Model': 'NearestCentroid', 'Accuracy': 0.7292817679558011, 'Balanced Accuracy': 0.7289987789987791, 'ROC AUC': 0.728998778998779, 'F1 Score': 0.7285357821965169, 'Time taken': 0.038858890533447266}
{'Model': 'NuSVC', 'Accuracy': 0.6629834254143646, 'Balanced Accuracy': 0.6623321123321123, 'ROC AUC': 0.6623321123321123, 'F1 Score': 0.6581595056740367, 'Time taken': 0.049308061599731445}
{'Model': 'PassiveAggressiveClassifier', 'Accuracy': 0.7016574585635359, 'Balanced Accuracy': 0.7007936507936507, 'ROC AUC': 0.7007936507936507, 'F1 Score': 0.6940680430357663, 'Time taken': 0.015070676803588867}
{'Model': 'Perceptron', 'Accuracy': 0.3314917127071823, 'Balanced Accuracy': 0.33235653235653234, 'ROC AUC': 0.3323565323565324, 'F1 Score': 0.3145161137053445, 'Time taken': 0.012476444244384

 97%|█████████▋| 28/29 [00:01<00:00, 25.97it/s]

{'Model': 'RandomForestClassifier', 'Accuracy': 0.6961325966850829, 'Balanced Accuracy': 0.6962148962148962, 'ROC AUC': 0.6962148962148963, 'F1 Score': 0.6960769382175832, 'Time taken': 0.23477458953857422}
{'Model': 'RidgeClassifier', 'Accuracy': 0.6961325966850829, 'Balanced Accuracy': 0.6962759462759462, 'ROC AUC': 0.6962759462759464, 'F1 Score': 0.6959470004698959, 'Time taken': 0.01751112937927246}
{'Model': 'RidgeClassifierCV', 'Accuracy': 0.6961325966850829, 'Balanced Accuracy': 0.6963369963369963, 'ROC AUC': 0.6963369963369963, 'F1 Score': 0.6957426064348392, 'Time taken': 0.012693166732788086}
{'Model': 'SGDClassifier', 'Accuracy': 0.6906077348066298, 'Balanced Accuracy': 0.6907814407814408, 'ROC AUC': 0.6907814407814408, 'F1 Score': 0.6903242008392605, 'Time taken': 0.012448549270629883}
{'Model': 'SVC', 'Accuracy': 0.7348066298342542, 'Balanced Accuracy': 0.7351037851037852, 'ROC AUC': 0.735103785103785, 'F1 Score': 0.7340762931355114, 'Time taken': 0.032325029373168945}
{'M

100%|██████████| 29/29 [00:01<00:00, 22.50it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [110]:
predict

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.73,0.74,0.74,0.73,0.03
GaussianNB,0.73,0.73,0.73,0.73,0.01
QuadraticDiscriminantAnalysis,0.73,0.73,0.73,0.73,0.01
NearestCentroid,0.73,0.73,0.73,0.73,0.04
AdaBoostClassifier,0.72,0.72,0.72,0.72,0.12
ExtraTreesClassifier,0.72,0.72,0.72,0.72,0.18
KNeighborsClassifier,0.71,0.71,0.71,0.71,0.02
LabelSpreading,0.71,0.71,0.71,0.71,0.05
LabelPropagation,0.71,0.71,0.71,0.71,0.04
